In [1]:
import separate_tree

In [2]:
from Bio import Phylo

In [3]:
import copy

In [4]:
from sklearn.utils import shuffle

import numpy as np

from sklearn import svm

from sklearn.naive_bayes import GaussianNB

In [5]:
from sklearn.naive_bayes import MultinomialNB

In [6]:
import itertools

In [7]:
import pandas as pd

In [8]:
import biom
import time

In [9]:
first_tic = time.time()

In [10]:
flag = 'DOG_IDB'
# test program
try:
    assert flag == 'test'
    biom_path = 'upload_files/feature-table.biom'
    tree_path = 'upload_files/tree.nwk'
    metadata_path = 'upload_files/demo_metadata.tsv'
    obj_col = 'BodySite'
    SampleID = '#SampleID'
except:
    # DOG_IDB data
    biom_path = 'Dog_IDB/all.biom'
    tree_path = 'Dog_IDB/tree.nwk'
    metadata_path = 'Dog_IDB/833_20180418-110621.txt'
    obj_col = 'disease_stat'
    SampleID = 'sample_name'

In [11]:
old_df = biom.load_table(biom_path).to_dataframe().T.to_dense()
old_df = old_df.divide(old_df.sum(axis=1),axis=0)

In [12]:
import time
tic = time.time()
mvp_tree = separate_tree.get_mvp_tree(biom_path,\
        tree_path,metadata_path)
new_tic = time.time()
# time used to gen mvp tree
print(new_tic-tic)

no #q2:types exist
2828.4004859924316


In [24]:
import pickle

In [25]:
with open('dog_idb_tree.pickle','wb') as f:
    pickle.dump(mvp_tree,f)

In [13]:
backup_mvp_tree = copy.deepcopy(mvp_tree.feature_tree)

In [14]:
init_terminals_num = len(backup_mvp_tree.get_terminals())

In [15]:
import corr_tree_new
import pandas as pd 
import copy 
import numpy as np 
import sklearn
def get_terminal_length(tree):
    terminal_lengths = {}
    for terminal in tree.get_terminals():
        path = tree.get_path(terminal)
        TBL = 0 # terminal branch length
        for ele in path:
            TBL += ele.branch_length
        terminal_lengths[terminal] = TBL
    sorted_terminal = sorted(terminal_lengths,key=terminal_lengths.get,reverse=True)
    sorted_terminal_lengths ={}
    for ele in sorted_terminal:
        sorted_terminal_lengths[ele] = terminal_lengths[ele]
   # print(sorted_terminal_lengths)
    return sorted_terminal_lengths

def cut_tree(tree,cp=0.2):
    """ 
    cut some branches of the tree.
    return the pruned branches.
    Args:
        tree: a Phylo.tree object
        cp: a percent used to cut the edge.
    """
    cp = 1-cp
    clades = []
    const = 1 # limit the terminal of a subtree
    terminal_lengths = get_terminal_length(tree)
    for terminal in terminal_lengths:
        current_terminals = tree.get_terminals()
        if not terminal in current_terminals:
            continue
        threshold = terminal_lengths[terminal]*cp
        temp = 0
        path = tree.get_path(terminal)
        for ele in path:
            temp += ele.branch_length
            if temp > threshold:
                #if not ele in current_terminals:  
                if len(ele.get_terminals()) > const:
                    backup = copy.copy(ele)
                    ele.clades = []
                    clades.append(backup)
                    try:
                        ele_parent = tree.get_path(ele)[-2]
                        ele_parent.clades.remove(ele)
                    except:
                        pass
                break
    return clades
def recurse_to_update(node):
    """used in the update sample series function.
    """
    if node.clades:
        node.sample_series = copy.deepcopy(recurse_to_update(node.clades[0]))
        for i,ele in enumerate(node.clades):
            if i == 0:
                continue
            else:
                node.sample_series += recurse_to_update(ele)
    else:
        return node.sample_series
def recursion_tree(node):
        """recursion to get the sample_series of a tree
        """
        if node.clades: # for non-leaf node
            tmp = 0
            flag = 0
            for clade in node.clades:
                if flag == 0:
                    tmp = copy.copy(recursion_tree(clade).sample_series)
                else:
                    tmp += recursion_tree(clade).sample_series   
                flag = 1
            node.sample_series = tmp
        else: # leaf node which has been init above.
            try:
                a = node.sample_series
                #print(node.name +' is a leaf')
            except:
                print('please initialize the tree leaves by otu table.')
        return node    
def update_sample_series(tree):
    """ update sample series after cut the tree.
    Args:
        tree: tree with sample_siries on the terminals.
    """
    recursion_tree(tree)
    #return tree
    
def get_mvp_tree(feature_table_path, tree_path, metadata_path):
    mvp_tree = corr_tree_new.MvpTree(feature_table_path, tree_path, metadata_path)
    return mvp_tree
def sep_mvp_tree(mvp_tree,cp):
    tree = copy.copy(mvp_tree.feature_tree)
    sub_trees = cut_tree(tree,cp)
    for ele in sub_trees:
        ele = update_sample_series(ele)
    print('len sub_trees:')
    print(len(sub_trees))
    return sub_trees

def get_node_score(node,coef=[1,1,1]):
    score = coef[0]*np.mean(node.sample_series)+\
        coef[1]*node.GI+coef[2]*np.std(node.sample_series)
    return score     

def search_sub_tree(sub_tree,coef,list1 = []):
    #print('search_sub_tree')
    p_score  = get_node_score(sub_tree,coef)
    child_score = 0
    for ele in sub_tree.clades:
        child_score += get_node_score(ele,coef)
    try:
        child_score = child_score/len(sub_tree.clades)
    except:
        pass
    if p_score <= child_score: 
        for ele in sub_tree.clades:
            search_sub_tree(ele,coef,list1)
    else:
        list1.append(sub_tree)
    #print('len list1')
    #print(len(list1))
    return list1

    
def generate_new_OTU_table(sub_trees,coef):
    """get a OTU table from the root node of the subtrees.
    Args:
        list1: obtained from the search_sub_tree function.
        coef: used for compute the score function,e.g.[1,1,1]
    """
    series = []
    for sub_tree in sub_trees:
        list1 = search_sub_tree(sub_tree,coef,list1=[])
        for ele in list1:
            series.append(ele.sample_series)
    df = pd.DataFrame(series)
    print('df.shape',df.shape)
    #print('df.index.name',df.index)
    return df

cp = 0.05
coef = [1,200,1]
sub_trees = sep_mvp_tree(mvp_tree,cp=cp)
df = generate_new_OTU_table(sub_trees,coef).T
mvp_tree.feature_tree =copy.deepcopy(backup_mvp_tree)

In [16]:
len(mvp_tree.feature_tree.get_terminals())

3434

In [17]:
#a

In [21]:
from sklearn.model_selection import cross_validate

from sklearn.ensemble import AdaBoostClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import GradientBoostingClassifier

In [18]:
#metadata.columns

In [19]:
metadata = pd.read_csv(metadata_path, sep= '\t')

metadata = metadata.set_index(SampleID)

In [22]:
feature_table_X = []
feature_table_Y = []
unpruned_clfs = {}
for ele in mvp_tree.feature_table.T.index:
    if metadata[obj_col][ele] == 'acute hem. diarrhea':
        continue
    feature_table_X.append(mvp_tree.feature_table.T.loc[ele])
    feature_table_Y.append(metadata[obj_col][ele])
            #clf = GradientBoostingClassifier(n_estimators=100,random_state=1) # 10 estimators 0.76 ,100 Ests 0.79
#old_clf = RandomForestClassifier(n_estimators=100, random_state=1) # 20 Ests.0.77, 100 Ests 0.79
            #clf = AdaBoostClassifier(random_state=1) # 0.73
            #clf = MultinomialNB(alpha=1)
#old_cv_result = cross_validate(old_clf,feature_table_X,feature_table_Y,cv=10,return_train_score=True)

In [104]:
feature_table_X = np.array(feature_table_X)

feature_table_Y = np.array(feature_table_Y)

(168, 3434)

In [130]:
skf1 = StratifiedKFold(n_splits=10,random_state = 1)
skf1.get_n_splits(feature_table_X,feature_table_Y)

10

In [131]:
old_clf = RandomForestClassifier(n_estimators=100, random_state=1)
aucs = []
fprs = []
tprs = []
for train_index, test_index in skf1.split(feature_table_X,feature_table_Y):
    X_train, X_test = feature_table_X[train_index], feature_table_X[test_index]
    y_train, y_test = feature_table_Y[train_index], feature_table_Y[test_index]
    old_clf.fit(X_train,y_train)
    y_pred = old_clf.predict_proba(X_test)[:,1]
    y_test = [[0,1][ele=='healthy'] for ele in y_test]
    auc = sklearn.metrics.roc_auc_score(y_test,y_pred)
    fpr, tpr, _ = roc_curve(y_test,y_pred)
    aucs.append(auc)
    fprs.append(fpr)
    tprs.append(tpr)
print(np.mean(aucs))

0.909117063492


In [141]:
aucs

[0.76875000000000004,
 0.86250000000000004,
 0.94999999999999996,
 0.84722222222222221,
 0.86111111111111116,
 0.92063492063492058,
 1.0,
 0.98412698412698418,
 0.97619047619047616,
 0.92063492063492069]

In [119]:
len(feature_table_Y)

168

In [27]:
categories = {}
flag = True
for ele in metadata[obj_col]:
    if ele in categories:
        categories[ele] += 1
    else:
        categories[ele] = 1
    if ele == 'acute hem. diarrhea' and flag:
        print(ele)
        flag = False
categories

acute hem. diarrhea


{'acute hem. diarrhea': 15, 'healthy': 98, 'IBD': 79}

In [35]:
#from sklearn.model_selection import StratifiedKFold

In [140]:
#f = open('sep_mvp_result.txt','w')
tic = time.time()
cps = [0.08]
#cps = [0.08]
#[1,800,200]
coefs = [#[1,20,1],
         #[1,15,1],
         #[1,10,7],
         #[1,1,7],
     #[1,1,10],
    [2,6,13]
     #[1,1,66]
       ]
acc_threshold = 0.5
continue_flag = True
cp_delta = 0.01
deltas = [1,2,3]
iter_num = 0
# up down bounds used to constrain the cp(cut percent) value.
up_bound = 0.15
down_bound = 0.01
# Max_key used to store the key.e.g str([0.08,1,20,1])
Max_key = ''
Last_Max_key = 'temp'
results = {}
#clfs = {}
while continue_flag:
    iter_num += 1
    for cp in cps:
        mvp_tree.feature_tree =copy.deepcopy(backup_mvp_tree)
        if len(mvp_tree.feature_tree.get_terminals()) < init_terminals_num:
            print('tree has been edited')
            break
        sub_trees = sep_mvp_tree(mvp_tree,cp=cp)
        for coef in coefs:
            if str([cp]+[coef]) in results:
                continue
            df = generate_new_OTU_table(sub_trees,coef).T
            X = []
            Y = []
            if df.shape == (0,0):
                continue
            for ele in df.index:
                if metadata[obj_col][ele] == 'acute hem. diarrhea':
                    continue
                X.append(df.loc[ele])
                Y.append(metadata[obj_col][ele])
            #clf = GradientBoostingClassifier(n_estimators=100,random_state=1) # 10 estimators 0.76 ,100 Ests 0.79
            #clf = RandomForestClassifier(n_estimators=100, random_state=1) # 20 Ests.0.77, 100 Ests 0.79
            #clf = AdaBoostClassifier(random_state=1) # 0.73
            #clf = MultinomialNB(alpha=1)
            #cv_result = cross_validate(clf,X,Y,cv=10,return_train_score=True)
            
            #temp_key = str([cp]+coef)
            #clfs[temp_key] = clf
            #results[temp_key]=cv_result
            X = np.array(X)
            y = np.array(Y)
            clf = RandomForestClassifier(n_estimators=100, random_state=1)
            cut_aucs = []
            cut_fprs = []
            cut_tprs = []
            accs = []
            for train_index, test_index in skf.split(X,y):
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]
                clf.fit(X_train,y_train)
                y_pred_prob = clf.predict_proba(X_test)[:,0]
                y_test = [[0,1][ele=='IBD'] for ele in y_test]
                auc = sklearn.metrics.roc_auc_score(y_test,y_pred_prob)
                y_pred = [[0,1][ele=='IBD'] for ele in clf.predict(X_test)]
                fpr, tpr, _ = roc_curve(y_test,y_pred_prob)
                cut_aucs.append(auc)
                cut_fprs.append(fpr)
                cut_tprs.append(tpr)
                assert len(y_test) ==len(y_pred)
                acc = sklearn.metrics.accuracy_score(y_pred,y_test)
                accs.append(acc)
            print(np.mean(cut_aucs))
            print(np.mean(accs))
    break
    """
   
    # generate new cps and coefs
    for key in results:
        if np.mean(results[key]['test_score']) > acc_threshold:
            acc_threshold = np.mean(results[key]['test_score'])
            Max_key = key
        else:
            pass
    if Max_key == Last_Max_key:
        print(Max_key)
        break
    Last_Max_key = copy.deepcopy(Max_key)
    Max_key_list = [float(ele) for ele in Max_key[1:-1].split(', ')]
    # update cps
    cps  = [max(Max_key_list[0]-cp_delta,down_bound), Max_key_list[0], min(Max_key_list[0]+cp_delta,up_bound)]
    # update coef
    Max_key_list.remove(Max_key_list[0])
    temps = [] # store paied coefs
    for i,ele  in enumerate(Max_key_list):
        temps.append([ele+deltas[i],ele-deltas[i]])
    coefs = list(itertools.product(temps[0],temps[1],temps[2]))
    coefs = [list(ele) for ele in coefs]
    #coef.append(temp)
    #print(results)
    print('cps',cps)
    print('coefs',coefs)
print('iter_num:',iter_num)
new_tic = time.time()
 """

len sub_trees:
161
df.shape (170, 182)
0.908898809524
0.87512254902


In [143]:
cut_aucs

[0.86874999999999991,
 0.85000000000000009,
 0.90000000000000002,
 0.86805555555555558,
 0.84027777777777779,
 0.92063492063492069,
 1.0,
 0.98412698412698407,
 0.93650793650793651,
 0.92063492063492069]

In [128]:
skf = StratifiedKFold(n_splits=10,random_state=1)
skf.get_n_splits(X,Y)

10

In [129]:
print(skf)

StratifiedKFold(n_splits=10, random_state=3, shuffle=False)


In [63]:
clf = RandomForestClassifier(n_estimators=100, random_state=1) 
#train_index,test_index = list(skf.split(X,Y))[0]

In [88]:
X = np.array(X)
y = np.array(Y)

In [67]:
#X_train, X_test = X[train_index], X[test_index]
#y_train, y_test = Y[train_index], Y[test_index]

In [103]:
#clf.fit(X_train,y_train)

(168, 132)

In [122]:
pred_y =clf.predict_proba(X_test)[:,1]

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [137]:
clf = RandomForestClassifier(n_estimators=100, random_state=1)
cut_aucs = []
cut_fprs = []
cut_tprs = []
accs = []
for train_index, test_index in skf.split(X,y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train,y_train)
    y_pred_prob = clf.predict_proba(X_test)[:,0]
    y_test = [[1,0][ele=='healthy'] for ele in y_test]
    auc = sklearn.metrics.roc_auc_score(y_test,y_pred_prob)
    y_pred = [[1,0][ele=='healthy'] for ele in clf.predict(X_test)]
    fpr, tpr, _ = roc_curve(y_test,y_pred_prob)
    cut_aucs.append(auc)
    cut_fprs.append(fpr)
    cut_tprs.append(tpr)
    assert len(y_test) ==len(y_pred)
    acc = sklearn.metrics.accuracy_score(y_pred,y_test)
    accs.append(acc)
print(np.mean(cut_aucs))
print(np.mean(accs))

0.906160714286
0.858455882353


In [77]:
y_test = [[-1,1][ele =='healthy'] for ele in y_test]

In [100]:
fpr, tpr,_ =sklearn.metrics.roc_curve(y_test, pred_y,pos_label=1,drop_intermediate=False)

In [102]:
auc = sklearn.metrics.roc_auc_score(y_test,pred_y)
auc

0.84375

In [30]:
print('total spend %.2f minutes' %((new_tic-tic)/60))

total spend 16.20 minutes


In [31]:
np.mean(results[Max_key]['test_score'])

0.87512254901960773

In [32]:
results[Max_key]['test_score']

array([ 0.83333333,  0.88888889,  0.94444444,  0.88235294,  0.76470588,
        0.8125    ,  1.        ,  0.9375    ,  0.875     ,  0.8125    ])

In [33]:
Max_key

'[0.08, 2.0, 6.0, 13.0]'

In [87]:
import numpy as np
from scipy import interp
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold

In [90]:
random_state = 0

In [91]:
cv = StratifiedKFold(n_splits=6)
classifier = svm.SVC(kernel='linear', probability=True,
                     random_state=random_state)

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

i = 0
for train, test in cv.split(X, y):
    probas_ = classifier.fit(X[train], y[train]).predict_proba(X[test])
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3,
             label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

    i += 1
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b',
         label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
         lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                 label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

/usr/lib/python3.6/site-packages/numpy/core/numeric.py:2604: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



ValueError: Data is not binary and pos_label is not specified